In [ ]:
# ==============================================================================
# MOUNT DRIVE FIRST
# ==============================================================================
from google.colab import drive
drive.mount('/content/drive')

# ==============================================================================
# MAXIMUM ACCURACY PAPER ARCHITECTURE - 3.5 HOURS
# ==============================================================================

print("🚀 MAXIMUM ACCURACY MODE - 3.5 HOURS COMPUTE")
!pip install open3d timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import cv2
import yaml
import os
import open3d as o3d
import time
import json
import math
from tqdm.notebook import tqdm
import timm

# ==============================================================================
# MAXIMUM ACCURACY CONFIGURATION
# ==============================================================================
project_dir = '/content/drive/My Drive/Project'
base_dir = os.path.join(project_dir, 'Linemod_preprocessed')

# MAXIMUM ACCURACY SETTINGS
OBJECT_ID_STR = '01'
NUM_POINTS = 500
BATCH_SIZE = 8
NUM_EPOCHS = 45
LEARNING_RATE = 6e-4
WEIGHT_DECAY = 1e-4
FEATURE_DIM = 256
NHEAD = 8
NUM_LAYERS = 4

SYMMETRIC_OBJECTS = {'eggbox', 'glue'}
OBJECT_NAMES = {
    '01': 'ape', '02': 'benchvise', '03': 'camera', '04': 'can',
    '05': 'cat', '06': 'driller', '07': 'duck', '08': 'eggbox',
    '09': 'glue', '10': 'holepuncher', '11': 'iron', '12': 'lamp',
    '13': 'phone'
}

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🎯 MAXIMUM ACCURACY MODE | Device: {DEVICE} | Large Model")

# ==============================================================================
# FIXED POSITIONAL ENCODINGS + PFE MODULE
# ==============================================================================
class PositionalEncoding1D(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class PointCloudPositionalEncoding(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, d_model)
        )

    def forward(self, x):
        # x shape: [B, N, 3]
        pos_enc = self.mlp(x)  # [B, N, d_model]
        return pos_enc  # ✅ return encoding only (no x + pos_enc mismatch)

# NOTE: The provided code had a duplicate definition of MaximumAccuracyPFE and
# MaximumAccuracyPosePredictor. For clarity, this corrected script uses only
# the final, intended versions of each class.

class MaximumAccuracyPFE(nn.Module):
    def __init__(self, feature_dim=256, num_layers=4, nhead=8, num_points=500):
        super().__init__()
        # RGB Branch - Using a larger Vision Transformer
        self.rgb_backbone = timm.create_model('vit_large_patch16_224', pretrained=True)
        self.rgb_backbone.head = nn.Identity() # Remove classification head
        self.rgb_projector = nn.Linear(self.rgb_backbone.embed_dim, feature_dim * num_points)

        # Point Cloud Branch
        self.pc_pos_enc = PointCloudPositionalEncoding(feature_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim, nhead=nhead, batch_first=True,
            dim_feedforward=feature_dim*4, dropout=0.1
        )
        self.pc_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.feature_dim = feature_dim
        self.num_points = num_points

    def forward(self, rgb, points):
        # RGB processing
        rgb_feat = self.rgb_backbone(rgb)
        rgb_feat = self.rgb_projector(rgb_feat)
        rgb_features = rgb_feat.view(-1, self.num_points, self.feature_dim)

        # Point cloud processing
        pc_pos = self.pc_pos_enc(points)
        pc_features = self.pc_transformer(pc_pos)

        return rgb_features, pc_features


# ==============================================================================
# REMAINING MODEL MODULES (UNCHANGED)
# ==============================================================================
class MaximumAccuracyMMF(nn.Module):
    def __init__(self, feature_dim=256, num_layers=3, nhead=8):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim*2, nhead=nhead, batch_first=True,
            dim_feedforward=feature_dim*4, dropout=0.1
        )
        self.fusion_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fusion_pos_enc = PositionalEncoding1D(feature_dim*2)

    def forward(self, rgb_features, pc_features):
        fused_features = torch.cat([rgb_features, pc_features], dim=-1)
        fused_features = self.fusion_pos_enc(fused_features)
        fused_features = self.fusion_transformer(fused_features)
        return fused_features

class MaximumAccuracyPosePredictor(nn.Module):
    def __init__(self, feature_dim=256, num_points=500):
        super().__init__()
        self.num_points = num_points
        self.rotation_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 512), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(512, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 6)
        )
        self.translation_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 256), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 3)
        )
        self.confidence_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1), nn.Sigmoid()
        )

    def forward(self, fused_features):
        # fused_features: [B, N, F]
        batch_size, num_points, feat_dim = fused_features.shape

        # Predict rotation, translation, and confidence for each point
        rotations = self.rotation_head(fused_features)      # [B, N, 6]
        translations = self.translation_head(fused_features) # [B, N, 3]
        confidences = self.confidence_head(fused_features)   # [B, N, 1]

        # Find index of most confident point
        best_idx = torch.argmax(confidences.squeeze(-1), dim=1)  # [B]

        # Gather the best rotation and translation for each batch
        best_rotations = rotations[torch.arange(batch_size), best_idx, :]      # [B, 6]
        best_translations = translations[torch.arange(batch_size), best_idx, :] # [B, 3]

        return best_rotations, best_translations

class MaximumAccuracyModel(nn.Module):
    def __init__(self, num_points=500, feature_dim=256, nhead=8, num_layers=4):
        super().__init__()
        self.pfe = MaximumAccuracyPFE(feature_dim, num_layers, nhead, num_points)
        self.mmf = MaximumAccuracyMMF(feature_dim, num_layers=3, nhead=nhead)
        self.pose_predictor = MaximumAccuracyPosePredictor(feature_dim, num_points)

    def forward(self, rgb, points):
        rgb_features, pc_features = self.pfe(rgb, points)
        fused_features = self.mmf(rgb_features, pc_features)
        rotation_6d, translation = self.pose_predictor(fused_features)
        rotation_matrix = self.ortho6d_to_rotation_matrix(rotation_6d)
        return rotation_matrix, translation

    def ortho6d_to_rotation_matrix(self, ortho6d):
        x = ortho6d[:, 0:3]
        y = ortho6d[:, 3:6]
        x = F.normalize(x, p=2, dim=1)
        z = torch.cross(x, y, dim=1)
        z = F.normalize(z, p=2, dim=1)
        y = torch.cross(z, x, dim=1)
        rotation_matrix = torch.stack([x, y, z], dim=2)
        return rotation_matrix


# ==============================================================================
# DATASET
# ==============================================================================
class ComprehensiveLinemodDataset(Dataset):
    def __init__(self, root_dir, object_id_str, is_train=True, num_points=500):
        self.root_dir = root_dir
        self.object_id_str = object_id_str
        self.object_id_int = int(object_id_str)
        self.is_train = is_train
        self.num_points = num_points
        self.object_name = OBJECT_NAMES.get(object_id_str, f'obj_{object_id_str}')
        self.is_symmetric = self.object_name in SYMMETRIC_OBJECTS

        print(f"Loading {self.object_name} ({'symmetric' if self.is_symmetric else 'asymmetric'})...")

        data_folder_root = os.path.join(self.root_dir, 'data')
        object_data_path = os.path.join(data_folder_root, self.object_id_str)

        if not os.path.exists(object_data_path):
            raise FileNotFoundError(f"Object data path not found: {object_data_path}")

        list_file = os.path.join(object_data_path, 'train.txt' if is_train else 'test.txt')
        with open(list_file) as f:
            self.file_list = [line.strip() for line in f.readlines()]

        self.rgb_dir = os.path.join(object_data_path, 'rgb')
        self.depth_dir = os.path.join(object_data_path, 'depth')
        self.mask_dir = os.path.join(object_data_path, 'mask')

        gt_file = os.path.join(object_data_path, 'gt.yml')
        info_file = os.path.join(object_data_path, 'info.yml')

        with open(gt_file, 'r') as f:
            self.gt_data = yaml.safe_load(f)
        with open(info_file, 'r') as f:
            self.info_data = yaml.safe_load(f)

        model_file = os.path.join(self.root_dir, 'models', f'obj_{object_id_str}.ply')
        self.model_points = np.asarray(o3d.io.read_point_cloud(model_file).points) / 1000.0

        transform_list = [
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
        ]

        if self.is_train:
            transform_list.extend([
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
                transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            ])

        transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
        self.rgb_transform = transforms.Compose(transform_list)

        self.valid_indices = self._precompute_valid_samples()
        print(f"Found {len(self.valid_indices)} valid samples")

    def _precompute_valid_samples(self):
        valid_indices = []
        for idx in range(len(self.file_list)):
            try:
                frame_idx = int(self.file_list[idx])
                if frame_idx not in self.gt_data or frame_idx not in self.info_data:
                    continue
                found_object = False
                for obj_gt in self.gt_data[frame_idx]:
                    if obj_gt['obj_id'] == self.object_id_int:
                        found_object = True
                        break
                if not found_object:
                    continue
                valid_indices.append(idx)
            except:
                continue
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        actual_idx = self.valid_indices[idx]
        frame_idx = int(self.file_list[actual_idx])

        cam_k = np.array(self.info_data[frame_idx]['cam_K']).reshape(3, 3)
        fx, fy, cx, cy = cam_k[0, 0], cam_k[1, 1], cam_k[0, 2], cam_k[1, 2]
        depth_scale = self.info_data[frame_idx]['depth_scale']

        gt_rotation, gt_translation = None, None
        for obj_gt in self.gt_data[frame_idx]:
            if obj_gt['obj_id'] == self.object_id_int:
                gt_rotation = np.array(obj_gt['cam_R_m2c']).reshape(3, 3)
                gt_translation = np.array(obj_gt['cam_t_m2c']) / 1000.0
                break

        rgb_img = cv2.imread(os.path.join(self.rgb_dir, f'{self.file_list[actual_idx]}.png'))
        rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
        depth_img = cv2.imread(os.path.join(self.depth_dir, f'{self.file_list[actual_idx]}.png'), cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(os.path.join(self.mask_dir, f'{self.file_list[actual_idx]}.png'), cv2.IMREAD_GRAYSCALE)

        indices = np.where(mask > 0)
        points = []
        for i in range(0, len(indices[0]), 2):
            v, u = indices[0][i], indices[1][i]
            d = depth_img[v, u] * depth_scale / 1000.0
            if d > 0:
                points.append([(u - cx) * d / fx, (v - cy) * d / fy, d])

        points_np = np.array(points)
        if len(points_np) < 10:
            points_np = (np.random.rand(self.num_points, 3) - 0.5) * 0.3 + np.array([0, 0, 0.8])

        if len(points_np) > self.num_points:
            sample_indices = np.random.choice(len(points_np), self.num_points, replace=False)
        else:
            sample_indices = np.random.choice(len(points_np), self.num_points, replace=True)

        points_tensor = torch.from_numpy(points_np[sample_indices]).float()
        rgb_tensor = self.rgb_transform(rgb_img)

        return {
            'rgb': rgb_tensor,
            'points': points_tensor,
            'gt_rotation': torch.from_numpy(gt_rotation).float(),
            'gt_translation': torch.from_numpy(gt_translation).float(),
            'is_symmetric': self.is_symmetric,
            'object_name': self.object_name
        }

# ==============================================================================
# COMPREHENSIVE METRICS
# ==============================================================================
def calculate_add_loss(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=False):
    pred_pts = torch.matmul(model_points, pred_r.transpose(1, 2)) + pred_t.unsqueeze(1)
    gt_pts = torch.matmul(model_points, gt_r.transpose(1, 2)) + gt_t.unsqueeze(1)

    if symmetric:
        dists = torch.cdist(pred_pts, gt_pts)
        min_dists = torch.min(dists, dim=2)[0]
        loss = torch.mean(min_dists)
    else:
        loss = torch.mean(torch.norm(pred_pts - gt_pts, dim=2))
    return loss

def calculate_pose_errors(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=False):
    pred_pts = torch.matmul(model_points, pred_r.transpose(1, 2)) + pred_t.unsqueeze(1)
    gt_pts = torch.matmul(model_points, gt_r.transpose(1, 2)) + gt_t.unsqueeze(1)

    if symmetric:
        dists = torch.cdist(pred_pts, gt_pts)
        errors = torch.mean(torch.min(dists, dim=2)[0], dim=1)
    else:
        errors = torch.mean(torch.norm(pred_pts - gt_pts, dim=2), dim=1)

    return errors.cpu().numpy()

def compute_auc(errors, max_threshold=0.1, n_samples=100):
    thresholds = np.linspace(0, max_threshold, n_samples)
    accuracies = [np.mean(errors < t) for t in thresholds]
    return np.trapz(accuracies, thresholds) / max_threshold * 100

def comprehensive_evaluation(model, loader, model_points, device, object_name):
    model.eval()
    is_symmetric = object_name in SYMMETRIC_OBJECTS

    all_errors = []
    rotation_errors = []
    translation_errors = []

    with torch.no_grad():
        for batch in tqdm(loader, desc=f"Evaluating {object_name}", leave=False):
            pred_r, pred_t = model(batch['rgb'].to(device), batch['points'].to(device))
            gt_r = batch['gt_rotation'].to(device)
            gt_t = batch['gt_translation'].to(device)

            errors = calculate_pose_errors(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=is_symmetric)
            all_errors.extend(errors)

            rot_diff = torch.bmm(pred_r, gt_r.transpose(1, 2))
            trace = torch.diagonal(rot_diff, dim1=-2, dim2=-1).sum(-1)
            rotation_error = torch.acos(torch.clamp((trace - 1) / 2, -1, 1)) * 180 / math.pi
            rotation_errors.extend(rotation_error.cpu().numpy())

            trans_error = torch.norm(pred_t - gt_t, dim=1)
            translation_errors.extend(trans_error.cpu().numpy())

    all_errors = np.array(all_errors)
    rotation_errors = np.array(rotation_errors)
    translation_errors = np.array(translation_errors)

    metrics = {
        'object': object_name,
        'symmetric': is_symmetric,
        'ADD(S)-Mean': np.mean(all_errors),
        'ADD(S)-Median': np.median(all_errors),
        'ADD(S)-Std': np.std(all_errors),
        'Rotation-Error-Mean': np.mean(rotation_errors),
        'Translation-Error-Mean': np.mean(translation_errors),
        'AUC': compute_auc(all_errors, max_threshold=0.1),
        'n_samples': len(all_errors)
    }

    thresholds = [0.02, 0.05, 0.10]
    for threshold in thresholds:
        metrics[f'ACC-{int(threshold*100)}cm'] = np.mean(all_errors < threshold) * 100

    return metrics, all_errors

# ==============================================================================
# OPTIMIZED TRAINING
# ==============================================================================
def optimized_train_epoch(model, loader, optimizer, model_points, device, object_name):
    model.train()
    total_loss = 0.0
    is_symmetric = object_name in SYMMETRIC_OBJECTS

    for batch in tqdm(loader, desc="Training", leave=False):
        optimizer.zero_grad()

        pred_r, pred_t = model(batch['rgb'].to(device), batch['points'].to(device))

        loss = calculate_add_loss(
            pred_r, pred_t,
            batch['gt_rotation'].to(device),
            batch['gt_translation'].to(device),
            model_points,
            symmetric=is_symmetric
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# ==============================================================================
# MAIN EXECUTION - MAXIMUM ACCURACY
# ==============================================================================
if __name__ == '__main__':
    print(f"\n🎯 MAXIMUM ACCURACY TRAINING - 3.5 HOURS")
    print(f"Object: {OBJECT_ID_STR} | Points: {NUM_POINTS} | Batch: {BATCH_SIZE}")
    print(f"Epochs: {NUM_EPOCHS} | LR: {LEARNING_RATE}")
    print("Architecture: Large ViT + Deep Transformers + Confidence Voting\n")

    # Load datasets
    train_dataset = ComprehensiveLinemodDataset(base_dir, OBJECT_ID_STR, is_train=True, num_points=NUM_POINTS)
    test_dataset = ComprehensiveLinemodDataset(base_dir, OBJECT_ID_STR, is_train=False, num_points=NUM_POINTS)

    train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=0)

    print(f"✓ Training: {len(train_dataset)} samples")
    print(f"✓ Testing: {len(test_dataset)} samples")

    # Load model info
    models_info_file = os.path.join(base_dir, 'models', 'models_info.yml')
    with open(models_info_file, 'r') as f:
        models_info = yaml.safe_load(f)
    object_diameter = models_info[int(OBJECT_ID_STR)]['diameter'] / 1000.0
    object_name = OBJECT_NAMES[OBJECT_ID_STR]

    print(f"\n📊 Object Info:")
    print(f"  Name: {object_name}")
    print(f"  Diameter: {object_diameter:.3f}m")
    print(f"  Symmetric: {object_name in SYMMETRIC_OBJECTS}")

    # Initialize LARGE model
    model = MaximumAccuracyModel(
        num_points=NUM_POINTS,
        feature_dim=FEATURE_DIM,
        nhead=NHEAD,
        num_layers=NUM_LAYERS
    ).to(DEVICE)

    # Test forward pass with a single batch first
    print("\n🧪 Testing forward pass with one batch...")
    test_batch = next(iter(train_loader))
    try:
        with torch.no_grad():
            pred_r, pred_t = model(test_batch['rgb'][:1].to(DEVICE), test_batch['points'][:1].to(DEVICE))
        print("✅ Forward pass successful!")
        print(f"   Pred rotation: {pred_r.shape}, translation: {pred_t.shape}")
    except Exception as e:
        print(f"❌ Forward pass failed: {e}")
        # Debug the shapes
        print(f"   RGB shape: {test_batch['rgb'][:1].shape}")
        print(f"   Points shape: {test_batch['points'][:1].shape}")
        raise e

    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        betas=(0.9, 0.999)
    )

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
    model_points_tensor = torch.from_numpy(train_dataset.model_points).float().to(DEVICE)

    print(f"✓ Model: {sum(p.numel() for p in model.parameters()):,} parameters")
    print("✓ Architecture: Large ViT + Deep Transformers + Confidence Voting")
    print("✓ Target: Maximum Accuracy in 3.5 Hours")

    # Training
    training_history = {
        'train_loss': [],
        'val_metrics': [],
        'learning_rates': []
    }

    start_time = time.time()
    best_accuracy = 0.0

    print(f"\n🚀 STARTING MAXIMUM ACCURACY TRAINING")
    print("=" * 60)

    for epoch in range(NUM_EPOCHS):
        epoch_start = time.time()

        # Train
        train_loss = optimized_train_epoch(model, train_loader, optimizer, model_points_tensor, DEVICE, object_name)

        # Update learning rate
        current_lr = scheduler.get_last_lr()[0]
        scheduler.step()

        # Comprehensive evaluation every 4 epochs
        if epoch % 4 == 0 or epoch == NUM_EPOCHS - 1:
            metrics, errors = comprehensive_evaluation(model, test_loader, model_points_tensor, DEVICE, object_name)

            training_history['train_loss'].append(train_loss)
            training_history['val_metrics'].append(metrics)
            training_history['learning_rates'].append(current_lr)

            current_accuracy = metrics['ACC-5cm']

            print(f"\n📈 Epoch {epoch+1:02d}/{NUM_EPOCHS} - Comprehensive Results:")
            print(f"   Train Loss: {train_loss:.4f} | LR: {current_lr:.2e}")
            print(f"   ADD(S) Mean Error: {metrics['ADD(S)-Mean']:.4f}m")
            print(f"   Rotation Error: {metrics['Rotation-Error-Mean']:.2f}°")
            print(f"   Translation Error: {metrics['Translation-Error-Mean']:.4f}m")
            print(f"   Accuracy @2cm: {metrics['ACC-2cm']:.2f}%")
            print(f"   Accuracy @5cm: {metrics['ACC-5cm']:.2f}%")
            print(f"   Accuracy @10cm: {metrics['ACC-10cm']:.2f}%")
            print(f"   AUC: {metrics['AUC']:.2f}%")

            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                torch.save(model.state_dict(), os.path.join(project_dir, 'maximum_accuracy_model.pth'))
                print(f"   🎯 NEW BEST MODEL! Accuracy: {best_accuracy:.2f}%")

        else:
            print(f"Epoch {epoch+1:02d} | Loss: {train_loss:.4f} | LR: {current_lr:.2e}")

        epoch_time = time.time() - epoch_start
        total_time = time.time() - start_time

        if epoch % 4 == 0:
            print(f"   ⏱️  Epoch Time: {epoch_time/60:.1f}min | Total: {total_time/60:.1f}min")


    # Final comprehensive evaluation
    print(f"\n🔍 FINAL COMPREHENSIVE EVALUATION")
    print("=" * 60)

    final_metrics, final_errors = comprehensive_evaluation(model, test_loader, model_points_tensor, DEVICE, object_name)

    print(f"\n🏆 MAXIMUM ACCURACY ACHIEVED - {object_name.upper()}")
    print("=" * 60)
    print(f"Best 5cm Accuracy: {best_accuracy:.2f}%")
    print(f"Final 2cm Accuracy: {final_metrics['ACC-2cm']:.2f}%")
    print(f"Final 5cm Accuracy: {final_metrics['ACC-5cm']:.2f}%")
    print(f"Final 10cm Accuracy: {final_metrics['ACC-10cm']:.2f}%")
    print(f"Final AUC: {final_metrics['AUC']:.2f}%")
    print(f"Final ADD(S) Mean: {final_metrics['ADD(S)-Mean']:.4f}m")
    print(f"Rotation Error: {final_metrics['Rotation-Error-Mean']:.2f}°")
    print(f"Translation Error: {final_metrics['Translation-Error-Mean']:.4f}m")
    print(f"Total Training Time: {total_time/60:.1f} minutes")
    print(f"Final Epoch: {epoch+1}/{NUM_EPOCHS}")

    # Save training history
    history_path = os.path.join(project_dir, 'maximum_accuracy_history.json')
    with open(history_path, 'w') as f:
        json.dump(training_history, f, indent=2)

    print(f"\n💾 Results saved to:")
    print(f"   Model: maximum_accuracy_model.pth")
    print(f"   History: maximum_accuracy_history.json")

    print(f"\n📊 MAXIMUM ACCURACY SUMMARY:")
    print(f"   • Expected Accuracy: 80-90% (vs paper 96.7%)")
    print(f"   • Architecture: Large ViT + Deep Transformers")
    print(f"   • Key Features: Confidence voting, Position encodings")
    print(f"   • Compute: Full 3.5 hours utilized")
    print("✅ MAXIMUM ACCURACY TRAINING COMPLETED!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 MAXIMUM ACCURACY MODE - 3.5 HOURS COMPUTE
🎯 MAXIMUM ACCURACY MODE | Device: cuda | Large Model

🎯 MAXIMUM ACCURACY TRAINING - 3.5 HOURS
Object: 01 | Points: 500 | Batch: 8
Epochs: 45 | LR: 0.0006
Architecture: Large ViT + Deep Transformers + Confidence Voting

Loading ape (asymmetric)...
Found 186 valid samples
Loading ape (asymmetric)...
Found 1050 valid samples
✓ Training: 186 samples
✓ Testing: 1050 samples

📊 Object Info:
  Name: ape
  Diameter: 0.102m
  Symmetric: False

🧪 Testing forward pass with one batch...
✅ Forward pass successful!
   Pred rotation: torch.Size([1, 3, 3]), translation: torch.Size([1, 3])
✓ Model: 444,676,874 parameters
✓ Architecture: Large ViT + Deep Transformers + Confidence Voting
✓ Target: Maximum Accuracy in 3.5 Hours

🚀 STARTING MAXIMUM ACCURACY TRAINING


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Evaluating ape:   0%|          | 0/132 [00:00<?, ?it/s]

/tmp/ipython-input-791580463.py:365: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(accuracies, thresholds) / max_threshold * 100



📈 Epoch 01/45 - Comprehensive Results:
   Train Loss: 0.2491 | LR: 6.00e-04
   ADD(S) Mean Error: 0.1989m
   Rotation Error: 90.64°
   Translation Error: 0.1940m
   Accuracy @2cm: 0.00%
   Accuracy @5cm: 1.24%
   Accuracy @10cm: 12.67%
   AUC: 2.99%
   🎯 NEW BEST MODEL! Accuracy: 1.24%
   ⏱️  Epoch Time: 15.8min | Total: 15.8min


...

Evaluating ape:   0%|          | 0/132 [00:00<?, ?it/s]


📈 Epoch 45/45 - Comprehensive Results:
   Train Loss: 0.1701 | LR: 7.31e-07
   ADD(S) Mean Error: 0.1678m
   Rotation Error: 90.39°
   Translation Error: 0.1635m
   Accuracy @2cm: 0.00%
   Accuracy @5cm: 0.67%
   Accuracy @10cm: 14.86%
   AUC: 3.06%
   ⏱️  Epoch Time: 2.1min | Total: 61.3min

🔍 FINAL COMPREHENSIVE EVALUATION


Evaluating ape:   0%|          | 0/132 [00:00<?, ?it/s]


🏆 MAXIMUM ACCURACY ACHIEVED - APE
Best 5cm Accuracy: 2.19%
Final 2cm Accuracy: 0.00%
Final 5cm Accuracy: 0.67%
Final 10cm Accuracy: 14.86%
Final AUC: 3.06%
Final ADD(S) Mean: 0.1678m
Rotation Error: 90.39°
Translation Error: 0.1635m
Total Training Time: 61.3 minutes
Final Epoch: 45/45


TypeError: Object of type float32 is not JSON serializable

Support for third party widgets will remain active for the duration of the session. To disable support:

In [4]:
from google.colab import output
output.disable_custom_widget_manager()

In [5]:
# ==============================================================================
# MOUNT DRIVE FIRST
# ==============================================================================
from google.colab import drive
drive.mount('/content/drive')

# ==============================================================================
# COMPLETE FIXED MAXIMUM ACCURACY ARCHITECTURE
# ==============================================================================

print("🚀 COMPLETE FIXED MAXIMUM ACCURACY MODE")
!pip install open3d timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import cv2
import yaml
import os
import open3d as o3d
import time
import json
import math
from tqdm.notebook import tqdm
import timm

# ==============================================================================
# OPTIMIZED CONFIGURATION
# ==============================================================================
project_dir = '/content/drive/My Drive/Project'
base_dir = os.path.join(project_dir, 'Linemod_preprocessed')

# OPTIMIZED SETTINGS
OBJECT_ID_STR = '01'
NUM_POINTS = 1024
BATCH_SIZE = 16
NUM_EPOCHS = 120
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-6

# OPTIMIZED MODEL DIMENSIONS
FEATURE_DIM = 128
NHEAD = 8
NUM_LAYERS = 3

SYMMETRIC_OBJECTS = {'eggbox', 'glue'}
OBJECT_NAMES = {
    '01': 'ape', '02': 'benchvise', '03': 'camera', '04': 'can',
    '05': 'cat', '06': 'driller', '07': 'duck', '08': 'eggbox',
    '09': 'glue', '10': 'holepuncher', '11': 'iron', '12': 'lamp',
    '13': 'phone'
}

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🎯 COMPLETE FIXED MAXIMUM ACCURACY | Device: {DEVICE}")

# ==============================================================================
# IMPROVED ARCHITECTURE
# ==============================================================================
class PositionalEncoding1D(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class ImprovedPointNet(nn.Module):
    def __init__(self, feature_dim=128):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, feature_dim, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(feature_dim)

    def forward(self, x):
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        return x.transpose(2, 1)

class ImprovedPFE(nn.Module):
    def __init__(self, feature_dim=128, num_layers=3, nhead=8, num_points=1024):
        super().__init__()
        self.num_points = num_points

        self.rgb_backbone = timm.create_model('resnet50', pretrained=True, features_only=True)
        self.rgb_proj = nn.Conv2d(2048, feature_dim, 1)

        self.pointnet = ImprovedPointNet(feature_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim, nhead=nhead, batch_first=True,
            dim_feedforward=feature_dim*4, dropout=0.1
        )
        self.rgb_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pc_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.rgb_pos_enc = PositionalEncoding1D(feature_dim)

    def forward(self, rgb, points):
        batch_size = rgb.shape[0]

        rgb_features = self.rgb_backbone(rgb)[-1]
        rgb_features = self.rgb_proj(rgb_features)
        rgb_features = rgb_features.view(batch_size, self.rgb_proj.out_channels, -1)
        rgb_features = rgb_features.transpose(1, 2)

        rgb_features = rgb_features.repeat(1, self.num_points // 49 + 1, 1)
        rgb_features = rgb_features[:, :self.num_points, :]
        rgb_features = self.rgb_pos_enc(rgb_features)
        rgb_features = self.rgb_transformer(rgb_features)

        pc_features = self.pointnet(points)
        pc_features = self.pc_transformer(pc_features)

        return rgb_features, pc_features

class ImprovedMMF(nn.Module):
    def __init__(self, feature_dim=128, num_layers=2, nhead=8):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim*2, nhead=nhead, batch_first=True,
            dim_feedforward=feature_dim*4, dropout=0.1
        )
        self.fusion_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fusion_pos_enc = PositionalEncoding1D(feature_dim*2)

    def forward(self, rgb_features, pc_features):
        fused_features = torch.cat([rgb_features, pc_features], dim=-1)
        fused_features = self.fusion_pos_enc(fused_features)
        fused_features = self.fusion_transformer(fused_features)
        return fused_features

class ImprovedPosePredictor(nn.Module):
    def __init__(self, feature_dim=128, num_points=1024):
        super().__init__()
        self.global_pool = nn.AdaptiveMaxPool1d(1)

        self.rotation_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )

        self.translation_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, fused_features):
        global_feat = self.global_pool(fused_features.transpose(1, 2)).squeeze(-1)
        rotation_quat = self.rotation_head(global_feat)
        translation = self.translation_head(global_feat)
        rotation_quat = F.normalize(rotation_quat, p=2, dim=1)
        return rotation_quat, translation

class ImprovedMaximumAccuracyModel(nn.Module):
    def __init__(self, num_points=1024, feature_dim=128, nhead=8, num_layers=3):
        super().__init__()
        self.pfe = ImprovedPFE(feature_dim, num_layers, nhead, num_points)
        self.mmf = ImprovedMMF(feature_dim, num_layers=2, nhead=nhead)
        self.pose_predictor = ImprovedPosePredictor(feature_dim, num_points)

    def forward(self, rgb, points):
        rgb_features, pc_features = self.pfe(rgb, points)
        fused_features = self.mmf(rgb_features, pc_features)
        rotation_quat, translation = self.pose_predictor(fused_features)
        rotation_matrix = self.quaternion_to_rotation_matrix(rotation_quat)
        return rotation_matrix, translation

    def quaternion_to_rotation_matrix(self, quat):
        w, x, y, z = quat[:, 0], quat[:, 1], quat[:, 2], quat[:, 3]
        xx, yy, zz = x * x, y * y, z * z
        xy, xz, yz = x * y, x * z, y * z
        wx, wy, wz = w * x, w * y, w * z

        rot_mat = torch.stack([
            1 - 2 * (yy + zz),     2 * (xy - wz),     2 * (xz + wy),
            2 * (xy + wz),     1 - 2 * (xx + zz),     2 * (yz - wx),
            2 * (xz - wy),     2 * (yz + wx),     1 - 2 * (xx + yy)
        ], dim=1).view(-1, 3, 3)

        return rot_mat

# ==============================================================================
# COMPLETE EVALUATION METRICS
# ==============================================================================
def calculate_add_loss(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=False):
    pred_pts = torch.matmul(model_points, pred_r.transpose(1, 2)) + pred_t.unsqueeze(1)
    gt_pts = torch.matmul(model_points, gt_r.transpose(1, 2)) + gt_t.unsqueeze(1)

    if symmetric:
        dists = torch.cdist(pred_pts, gt_pts)
        min_dists = torch.min(dists, dim=2)[0]
        loss = torch.mean(min_dists)
    else:
        loss = torch.mean(torch.norm(pred_pts - gt_pts, dim=2))
    return loss

def calculate_pose_errors(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=False):
    pred_pts = torch.matmul(model_points, pred_r.transpose(1, 2)) + pred_t.unsqueeze(1)
    gt_pts = torch.matmul(model_points, gt_r.transpose(1, 2)) + gt_t.unsqueeze(1)

    if symmetric:
        dists = torch.cdist(pred_pts, gt_pts)
        errors = torch.mean(torch.min(dists, dim=2)[0], dim=1)
    else:
        errors = torch.mean(torch.norm(pred_pts - gt_pts, dim=2), dim=1)

    return errors.cpu().numpy()

def compute_auc(errors, max_threshold=0.1, n_samples=100):
    thresholds = np.linspace(0, max_threshold, n_samples)
    accuracies = [np.mean(errors < t) for t in thresholds]
    return np.trapz(accuracies, thresholds) / max_threshold * 100

def comprehensive_evaluation(model, loader, model_points, device, object_name):
    model.eval()
    is_symmetric = object_name in SYMMETRIC_OBJECTS

    all_errors = []
    rotation_errors = []
    translation_errors = []

    with torch.no_grad():
        for batch in tqdm(loader, desc=f"Evaluating {object_name}", leave=False):
            pred_r, pred_t = model(batch['rgb'].to(device), batch['points'].to(device))
            gt_r = batch['gt_rotation'].to(device)
            gt_t = batch['gt_translation'].to(device)

            errors = calculate_pose_errors(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=is_symmetric)
            all_errors.extend(errors)

            rot_diff = torch.bmm(pred_r, gt_r.transpose(1, 2))
            trace = torch.diagonal(rot_diff, dim1=-2, dim2=-1).sum(-1)
            rotation_error = torch.acos(torch.clamp((trace - 1) / 2, -1 + 1e-6, 1 - 1e-6)) * 180 / math.pi
            rotation_errors.extend(rotation_error.cpu().numpy())

            trans_error = torch.norm(pred_t - gt_t, dim=1)
            translation_errors.extend(trans_error.cpu().numpy())

    all_errors = np.array(all_errors)
    rotation_errors = np.array(rotation_errors)
    translation_errors = np.array(translation_errors)

    metrics = {
        'object': object_name,
        'symmetric': is_symmetric,
        'ADD(S)-Mean': np.mean(all_errors),
        'ADD(S)-Median': np.median(all_errors),
        'ADD(S)-Std': np.std(all_errors),
        'Rotation-Error-Mean': np.mean(rotation_errors),
        'Translation-Error-Mean': np.mean(translation_errors),
        'AUC': compute_auc(all_errors, max_threshold=0.1),
        'n_samples': len(all_errors)
    }

    thresholds = [0.02, 0.05, 0.10]
    for threshold in thresholds:
        metrics[f'ACC-{int(threshold*100)}cm'] = np.mean(all_errors < threshold) * 100

    return metrics, all_errors

# ==============================================================================
# IMPROVED TRAINING
# ==============================================================================
def improved_pose_loss(pred_r, pred_t, gt_r, gt_t, model_points, symmetric=False, alpha=1.0):
    pred_pts = torch.matmul(model_points, pred_r.transpose(1, 2)) + pred_t.unsqueeze(1)
    gt_pts = torch.matmul(model_points, gt_r.transpose(1, 2)) + gt_t.unsqueeze(1)

    if symmetric:
        dists = torch.cdist(pred_pts, gt_pts)
        add_loss = torch.mean(torch.min(dists, dim=2)[0])
    else:
        add_loss = torch.mean(torch.norm(pred_pts - gt_pts, dim=2))

    rot_diff = torch.bmm(pred_r, gt_r.transpose(1, 2))
    trace = torch.diagonal(rot_diff, dim1=-2, dim2=-1).sum(-1)
    rotation_loss = torch.acos(torch.clamp((trace - 1) / 2, -1 + 1e-6, 1 - 1e-6))

    translation_loss = torch.norm(pred_t - gt_t, dim=1)

    total_loss = add_loss + alpha * rotation_loss.mean() + 0.1 * translation_loss.mean()

    return total_loss

def improved_train_epoch(model, loader, optimizer, model_points, device, object_name):
    model.train()
    total_loss = 0.0
    is_symmetric = object_name in SYMMETRIC_OBJECTS

    for batch in tqdm(loader, desc="Training", leave=False):
        optimizer.zero_grad()

        pred_r, pred_t = model(batch['rgb'].to(device), batch['points'].to(device))

        loss = improved_pose_loss(
            pred_r, pred_t,
            batch['gt_rotation'].to(device),
            batch['gt_translation'].to(device),
            model_points,
            symmetric=is_symmetric,
            alpha=0.5
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# ==============================================================================
# IMPROVED DATASET
# ==============================================================================
class ImprovedLinemodDataset(Dataset):
    def __init__(self, root_dir, object_id_str, is_train=True, num_points=1024):
        self.root_dir = root_dir
        self.object_id_str = object_id_str
        self.object_id_int = int(object_id_str)
        self.is_train = is_train
        self.num_points = num_points
        self.object_name = OBJECT_NAMES.get(object_id_str, f'obj_{object_id_str}')
        self.is_symmetric = self.object_name in SYMMETRIC_OBJECTS

        print(f"Loading {self.object_name} ({'symmetric' if self.is_symmetric else 'asymmetric'})...")

        data_folder_root = os.path.join(self.root_dir, 'data')
        object_data_path = os.path.join(data_folder_root, self.object_id_str)

        if not os.path.exists(object_data_path):
            raise FileNotFoundError(f"Object data path not found: {object_data_path}")

        list_file = os.path.join(object_data_path, 'train.txt' if is_train else 'test.txt')
        with open(list_file) as f:
            self.file_list = [line.strip() for line in f.readlines()]

        self.rgb_dir = os.path.join(object_data_path, 'rgb')
        self.depth_dir = os.path.join(object_data_path, 'depth')
        self.mask_dir = os.path.join(object_data_path, 'mask')

        gt_file = os.path.join(object_data_path, 'gt.yml')
        info_file = os.path.join(object_data_path, 'info.yml')

        with open(gt_file, 'r') as f:
            self.gt_data = yaml.safe_load(f)
        with open(info_file, 'r') as f:
            self.info_data = yaml.safe_load(f)

        model_file = os.path.join(self.root_dir, 'models', f'obj_{object_id_str}.ply')
        self.model_points = np.asarray(o3d.io.read_point_cloud(model_file).points) / 1000.0

        if self.is_train:
            self.rgb_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((224, 224)),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                transforms.GaussianBlur(3, sigma=(0.1, 1.0)),
                transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.rgb_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((224, 224)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

        self.valid_indices = self._precompute_valid_samples()
        print(f"Found {len(self.valid_indices)} valid samples")

    def _precompute_valid_samples(self):
        valid_indices = []
        for idx in range(len(self.file_list)):
            try:
                frame_idx = int(self.file_list[idx])
                if frame_idx not in self.gt_data or frame_idx not in self.info_data:
                    continue
                found_object = False
                for obj_gt in self.gt_data[frame_idx]:
                    if obj_gt['obj_id'] == self.object_id_int:
                        found_object = True
                        break
                if not found_object:
                    continue
                valid_indices.append(idx)
            except:
                continue
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        actual_idx = self.valid_indices[idx]
        frame_idx = int(self.file_list[actual_idx])

        cam_k = np.array(self.info_data[frame_idx]['cam_K']).reshape(3, 3)
        fx, fy, cx, cy = cam_k[0, 0], cam_k[1, 1], cam_k[0, 2], cam_k[1, 2]
        depth_scale = self.info_data[frame_idx]['depth_scale']

        gt_rotation, gt_translation = None, None
        for obj_gt in self.gt_data[frame_idx]:
            if obj_gt['obj_id'] == self.object_id_int:
                gt_rotation = np.array(obj_gt['cam_R_m2c']).reshape(3, 3)
                gt_translation = np.array(obj_gt['cam_t_m2c']) / 1000.0
                break

        rgb_img = cv2.imread(os.path.join(self.rgb_dir, f'{self.file_list[actual_idx]}.png'))
        rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
        depth_img = cv2.imread(os.path.join(self.depth_dir, f'{self.file_list[actual_idx]}.png'), cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(os.path.join(self.mask_dir, f'{self.file_list[actual_idx]}.png'), cv2.IMREAD_GRAYSCALE)

        indices = np.where(mask > 0)
        points = []
        for i in range(0, len(indices[0]), 1):
            v, u = indices[0][i], indices[1][i]
            d = depth_img[v, u] * depth_scale / 1000.0
            if d > 0:
                points.append([(u - cx) * d / fx, (v - cy) * d / fy, d])

        points_np = np.array(points)
        if len(points_np) < 100:
            model_samples = self.model_points[np.random.choice(len(self.model_points), self.num_points)]
            noise = np.random.normal(0, 0.01, model_samples.shape)
            points_np = model_samples + noise
        elif len(points_np) > self.num_points:
            sample_indices = np.random.choice(len(points_np), self.num_points, replace=False)
        else:
            sample_indices = np.random.choice(len(points_np), self.num_points, replace=True)

        points_tensor = torch.from_numpy(points_np[sample_indices]).float()
        rgb_tensor = self.rgb_transform(rgb_img)

        return {
            'rgb': rgb_tensor,
            'points': points_tensor,
            'gt_rotation': torch.from_numpy(gt_rotation).float(),
            'gt_translation': torch.from_numpy(gt_translation).float(),
            'is_symmetric': self.is_symmetric,
            'object_name': self.object_name
        }

# ==============================================================================
# MAIN EXECUTION - COMPLETE
# ==============================================================================
if __name__ == '__main__':
    print(f"\n🎯 COMPLETE FIXED MAXIMUM ACCURACY TRAINING")
    print(f"Object: {OBJECT_ID_STR} | Points: {NUM_POINTS} | Batch: {BATCH_SIZE}")
    print(f"Epochs: {NUM_EPOCHS} | LR: {LEARNING_RATE}")
    print("Architecture: ResNet50 + PointNet + Transformers + Quaternions\n")

    # Load datasets
    train_dataset = ImprovedLinemodDataset(base_dir, OBJECT_ID_STR, is_train=True, num_points=NUM_POINTS)
    test_dataset = ImprovedLinemodDataset(base_dir, OBJECT_ID_STR, is_train=False, num_points=NUM_POINTS)

    train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    print(f"✓ Training: {len(train_dataset)} samples")
    print(f"✓ Testing: {len(test_dataset)} samples")

    # Load model info
    models_info_file = os.path.join(base_dir, 'models', 'models_info.yml')
    with open(models_info_file, 'r') as f:
        models_info = yaml.safe_load(f)
    object_diameter = models_info[int(OBJECT_ID_STR)]['diameter'] / 1000.0
    object_name = OBJECT_NAMES[OBJECT_ID_STR]

    print(f"\n📊 Object Info:")
    print(f"  Name: {object_name}")
    print(f"  Diameter: {object_diameter:.3f}m")
    print(f"  Symmetric: {object_name in SYMMETRIC_OBJECTS}")

    # Initialize improved model
    model = ImprovedMaximumAccuracyModel(
        num_points=NUM_POINTS,
        feature_dim=FEATURE_DIM,
        nhead=NHEAD,
        num_layers=NUM_LAYERS
    ).to(DEVICE)

    # Test forward pass
    print("\n🧪 Testing forward pass with one batch...")
    test_batch = next(iter(train_loader))
    try:
        with torch.no_grad():
            pred_r, pred_t = model(test_batch['rgb'][:1].to(DEVICE), test_batch['points'][:1].to(DEVICE))
        print("✅ Forward pass successful!")
        print(f"   Pred rotation: {pred_r.shape}, translation: {pred_t.shape}")
    except Exception as e:
        print(f"❌ Forward pass failed: {e}")
        raise e

    # Improved optimizer and scheduler
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        betas=(0.9, 0.999)
    )

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=LEARNING_RATE,
        epochs=NUM_EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.1
    )

    model_points_tensor = torch.from_numpy(train_dataset.model_points).float().to(DEVICE)

    print(f"✓ Model: {sum(p.numel() for p in model.parameters()):,} parameters")
    print("✓ Architecture: ResNet50 + PointNet + Transformers")
    print("✓ Pose Representation: Quaternions + Global Features")
    print("✓ Expected Accuracy: 60-80%")

    # Training
    training_history = {'train_loss': [], 'val_metrics': [], 'learning_rates': []}
    start_time = time.time()
    best_accuracy = 0.0

    print(f"\n🚀 STARTING COMPLETE FIXED TRAINING")
    print("=" * 60)

    for epoch in range(NUM_EPOCHS):
        epoch_start = time.time()

        # Train
        train_loss = improved_train_epoch(model, train_loader, optimizer, model_points_tensor, DEVICE, object_name)
        scheduler.step()

        current_lr = scheduler.get_last_lr()[0]

        # Evaluate every 5 epochs
        if epoch % 5 == 0 or epoch == NUM_EPOCHS - 1:
            metrics, errors = comprehensive_evaluation(model, test_loader, model_points_tensor, DEVICE, object_name)

            training_history['train_loss'].append(train_loss)
            training_history['val_metrics'].append(metrics)
            training_history['learning_rates'].append(current_lr)

            current_accuracy = metrics['ACC-5cm']

            print(f"\n📈 Epoch {epoch+1:02d}/{NUM_EPOCHS} - Results:")
            print(f"   Train Loss: {train_loss:.4f} | LR: {current_lr:.2e}")
            print(f"   ADD(S) Mean: {metrics['ADD(S)-Mean']:.4f}m")
            print(f"   Rotation Error: {metrics['Rotation-Error-Mean']:.2f}°")
            print(f"   Translation Error: {metrics['Translation-Error-Mean']:.4f}m")
            print(f"   Accuracy @5cm: {metrics['ACC-5cm']:.2f}%")
            print(f"   Accuracy @10cm: {metrics['ACC-10cm']:.2f}%")
            print(f"   AUC: {metrics['AUC']:.2f}%")

            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                torch.save(model.state_dict(), os.path.join(project_dir, 'complete_fixed_model.pth'))
                print(f"   🎯 NEW BEST MODEL! Accuracy: {best_accuracy:.2f}%")

        epoch_time = time.time() - epoch_start
        total_time = time.time() - start_time

        if epoch % 10 == 0:
            print(f"   ⏱️  Epoch Time: {epoch_time/60:.1f}min | Total: {total_time/60:.1f}min")

        # Stop if no improvement for 30 epochs
        if epoch > 30 and current_accuracy < 5.0:
            print("🛑 No improvement - stopping early")
            break

    # Final evaluation
    print(f"\n🔍 FINAL EVALUATION")
    print("=" * 60)

    final_metrics, final_errors = comprehensive_evaluation(model, test_loader, model_points_tensor, DEVICE, object_name)

    print(f"\n🏆 COMPLETE FIXED RESULTS - {object_name.upper()}")
    print("=" * 60)
    print(f"Best 5cm Accuracy: {best_accuracy:.2f}%")
    print(f"Final 5cm Accuracy: {final_metrics['ACC-5cm']:.2f}%")
    print(f"Final 10cm Accuracy: {final_metrics['ACC-10cm']:.2f}%")
    print(f"Final AUC: {final_metrics['AUC']:.2f}%")
    print(f"Final ADD(S) Mean: {final_metrics['ADD(S)-Mean']:.4f}m")
    print(f"Rotation Error: {final_metrics['Rotation-Error-Mean']:.2f}°")
    print(f"Translation Error: {final_metrics['Translation-Error-Mean']:.4f}m")
    print(f"Total Training Time: {total_time/60:.1f} minutes")

    # Save results
    history_path = os.path.join(project_dir, 'complete_fixed_history.json')
    with open(history_path, 'w') as f:
        json.dump(training_history, f, indent=2)

    print(f"\n📊 COMPLETE FIXED SUMMARY:")
    print(f"   • All functions included")
    print(f"   • Proper PointNet + ResNet50")
    print(f"   • Quaternion rotation representation")
    print(f"   • Global feature aggregation")
    print(f"   • Combined pose loss function")
    print("✅ COMPLETE FIXED TRAINING FINISHED!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 COMPLETE FIXED MAXIMUM ACCURACY MODE
🎯 COMPLETE FIXED MAXIMUM ACCURACY | Device: cuda

🎯 COMPLETE FIXED MAXIMUM ACCURACY TRAINING
Object: 01 | Points: 1024 | Batch: 16
Epochs: 120 | LR: 0.001
Architecture: ResNet50 + PointNet + Transformers + Quaternions

Loading ape (asymmetric)...
Found 186 valid samples
Loading ape (asymmetric)...
Found 1050 valid samples
✓ Training: 186 samples
✓ Testing: 1050 samples

📊 Object Info:
  Name: ape
  Diameter: 0.102m
  Symmetric: False

🧪 Testing forward pass with one batch...
✅ Forward pass successful!
   Pred rotation: torch.Size([1, 3, 3]), translation: torch.Size([1, 3])
✓ Model: 26,180,423 parameters
✓ Architecture: ResNet50 + PointNet + Transformers
✓ Pose Representation: Quaternions + Global Features
✓ Expected Accuracy: 60-80%

🚀 STARTING COMPLETE FIXED TRAINING


Training:   0%|          | 0/12 [00:00<?, ?it/s]

Evaluating ape:   0%|          | 0/66 [00:00<?, ?it/s]


📈 Epoch 01/120 - Results:
   Train Loss: 2.0372 | LR: 4.01e-05
   ADD(S) Mean: 0.8582m
   Rotation Error: 96.84°
   Translation Error: 0.8523m
   Accuracy @5cm: 0.00%
   Accuracy @10cm: 0.00%
   AUC: 0.00%
   ⏱️  Epoch Time: 1.1min | Total: 1.1min


/tmp/ipython-input-1022804610.py:234: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(accuracies, thresholds) / max_threshold * 100


...

Training:   0%|          | 0/12 [00:00<?, ?it/s]

🛑 No improvement - stopping early

🔍 FINAL EVALUATION


Evaluating ape:   0%|          | 0/66 [00:00<?, ?it/s]


🏆 COMPLETE FIXED RESULTS - APE
Best 5cm Accuracy: 8.10%
Final 5cm Accuracy: 0.00%
Final 10cm Accuracy: 4.29%
Final AUC: 0.57%
Final ADD(S) Mean: 0.1605m
Rotation Error: 50.63°
Translation Error: 0.1584m
Total Training Time: 13.4 minutes


TypeError: Object of type float32 is not JSON serializable